In [1]:
import pandas as pd
import csv
from datetime import datetime
from pandas.tseries.offsets import BDay
from openpyxl import load_workbook, Workbook
import math
import os
import shutil
import numpy as np
import pdfplumber
from pathlib import Path
import itertools

In [2]:
folder_name = 'past'
os.makedirs(folder_name, exist_ok=True)
files = os.listdir()
for file in files:
    if ('xlsx' in file or 'xls' in file or 'csv' in file or 'pdf' in file):#and ('sedol' not in file)
        shutil.move(file, os.path.join(folder_name, file))  

In [3]:
pd.set_option('display.max_rows',200)

In [4]:
yDay = datetime.now()-BDay(2)
date4=yDay.strftime("%Y.%m.%d")
currentFolder = os.getcwd()
sourceFolder=r"C:\Users\Ning\Seahawk China Dynamic Fund\Seahawk Operations - Documents\portfolio"
holdingReportPath='Seahawk portfolio report '+date4+ '.xlsm'
hrPath=Path(r"C:\Users\Ning\Seahawk China Dynamic Fund\Seahawk Operations - Documents\portfolio\Holding reports") / holdingReportPath
shutil.copy(hrPath, currentFolder)

'C:\\Users\\Ning\\Daily recap 1213\\margin_borrow\\Seahawk portfolio report 2025.04.08.xlsm'

In [5]:
sedol_form=pd.read_excel(holdingReportPath, sheet_name="PV Bloomberg", skiprows=1)

In [6]:
OBFR=sedol_form['PX_LAST'][sedol_form['Ticker']=='OBFR'].iloc[0]
HIBOR1M=sedol_form['PX_LAST'][sedol_form['Ticker']=='HIBOR1M'].iloc[0]
rate_dict={
    "US": OBFR,
    'HK': HIBOR1M
}

In [7]:
sedol_form=sedol_form.loc[:,['Ticker','ID_SEDOL1']]
sedol_form.dropna(inplace=True)
sedol_form['ID_SEDOL1']=sedol_form['ID_SEDOL1'].astype(str)
sedol_form.set_index('ID_SEDOL1',inplace=True)
sedol_dict=sedol_form['Ticker'].to_dict()

In [8]:
sedol_dict['A057341']='9956 HK'
sedol_dict['H089610']='1179 HK'
sedol_dict['XXJ059253']='9636 HK'
sedol_dict['XXG010208']='2333 HK'
sedol_dict['XXC066320']='1113 HK'
sedol_dict['XXC055217']='1908 HK'
sedol_dict['XXR020014']='1196 HK'
sedol_dict['XXY002114']='1818 HK'

In [9]:
class Broker(object):
    
    def __init__(self, folder, mfile,*vfile):
        self.name=self.__class__.__name__
        self.folder=sourceFolder+"\\"+folder
        
        if folder!="GF":
            self.mfiles=[file for file in os.listdir(self.folder) if mfile in file]
        else:
            self.mfiles=[file for file in os.listdir(self.folder) if mfile in file and 'Trade' not in file]      
        self.mPath=self.mfiles[-1]
        self.mSourcePath=os.path.join(self.folder,self.mPath)
        self.mDestinationPath=os.path.join(currentFolder,self.mPath)
        shutil.copy(self.mSourcePath,self.mDestinationPath)

        if len(vfile)>0:
            self.vfiles=[file for file in os.listdir(self.folder) if vfile[0] in file]
            self.vPath=self.vfiles[-1]
        else:
            self.vPath=self.mPath
        self.vSourcePath=os.path.join(self.folder,self.vPath)
        self.vDestinationPath=os.path.join(currentFolder,self.vPath)
        shutil.copy(self.vSourcePath,self.vDestinationPath)

        #if len(vfile)>1:
        #    for i in range(1,len(vfile)):
        #        self.afiles=[file for file in os.listdir(self.folder) if vfile[i] in file]
        #        self.aPath=self.afiles[-1]
        #        self.aSourcePath=os.path.join(self.folder,self.aPath)
        #        self.aDestinationPath=os.path.join(currentFolder,self.aPath)
        #        shutil.copy(self.aSourcePath,self.aDestinationPath)
                
    def get_margin(self):
        self.margin=pd.DataFrame(columns=['symbol','margin'])
        return self.margin
    
    def margin_summary(self,df):
        margin=df.groupby('symbol').agg({'margin':'mean'}).reset_index()        
        margin['margin']=margin['margin'].map(lambda x: '{:.1%}'.format(x))
        marginColumn=self.name+'_margin'
        margin.columns=['symbol',marginColumn]  
        return margin
    
    def get_financing(self):
        self.financing=pd.DataFrame(columns=['symbol','broker','benchmark','spread'])
        return self.financing
    
    def financing_summary(self, df):
        financingSummary=df.groupby('symbol').agg({'benchmark':'last','spread':'last'}).reset_index()        
        financingSummary['spread']=financingSummary['spread'].map(lambda x: '{:.2%}'.format(x))
        benchmarkColumn=self.name+'_benchmark'
        spreadColumn=self.name+'_spread'
        financingSummary.columns=['symbol',benchmarkColumn,spreadColumn]  
        return financingSummary        
    

In [10]:
def macq_symbol(symbol):
    symbol=symbol.replace("."," ").replace(" OQ"," US").replace(" N"," US").replace(' SH',' C1').replace(' ZK',' C2').replace(' TWO',' TT').replace(' TW',' TT').replace(' T',' JP').replace(' JPT',' TT')
    if " C2" not in symbol:
        symbol=symbol.lstrip("0")
    symbol=symbol.replace("YAZG SI","YZJFH SP")
    return symbol

class Macq(Broker):
    
    def get_margin(self):
        df=pd.read_excel(self.mPath, sheet_name='ps-position-report',skiprows=10)
        df=df[df["Ticker"].notna()]
        df=df.loc[:,['Ticker','Initial Margin %']]
        df.columns=['symbol','margin']
        df['symbol']=df['symbol'].map(macq_symbol)
        df['margin']=df['margin']/100

        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_excel(self.vPath, sheet_name='ps-position-report',skiprows=10)
        df=df[df["Ticker"].notna()]
        df=df.loc[:,['Ticker','Swap Ccy','Average Spread %','Qty']]
        df.columns=['symbol','benchmark','spread','qty']
        df['symbol']=df['symbol'].map(macq_symbol)
        df['benchmark']=df['benchmark'].map({'HKD':'HIBOR1M','USD':'SOFRON'})
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']]
        return self.financing

In [11]:
class Clsa(Broker):
    
    def get_margin(self):
        df=pd.read_excel(self.mPath, sheet_name='Position Margin',skiprows=4)
        df=df[df['Type']!='Pay-to-hold']
        df=df.loc[:,['Underlying Security Name','Independent Amount %']]
        df.columns=['symbol','margin']
        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_excel(self.vPath, sheet_name='Position valuation report',skiprows=4)
        df=df.loc[:,['Underlying Security Name','Rate Index','Spread','Quantity / Exchange-traded Contract']]
        df.columns=['symbol','benchmark','spread','qty']
        df['spread']=df['spread']*df['qty'].map(lambda x:np.sign(x))
        df['benchmark']=df['benchmark'].map(lambda x: x.replace('USD/SOFR/1D/SOFR','SOFRON').replace('USD/SOFR-REBATE/1D/SOFR','SOFRON').replace('HKD/HIBOR/1M/HKAB','HIBOR1M').replace('HKD/HIBOR-REBATE/1D/HKAB','HIBORON') if isinstance(x,str) else x)
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']]
        return self.financing    

In [12]:
class Cicc(Broker): 
    
    def get_margin(self):
        df=pd.read_excel(self.mPath, sheet_name='Valuation',skiprows=11)
        df=df[df["Bloomberg Code"].notna()]
        df=df.loc[:,['Bloomberg Code','Initial Margin Rate']]
        df.columns=['symbol','margin']
        df['symbol']=df['symbol'].astype(str).map(lambda x: x.replace(" Equity","").replace(" EQUITY",""))
        df['margin']=df['margin'].map(lambda x: float(x.replace('%',"") if isinstance(x,str) else x*100))
        df['margin']=df['margin']/100
        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_excel(self.vPath, sheet_name='Valuation',skiprows=11)
        df=df[df["Bloomberg Code"].notna()]
        df=df.loc[:,['Bloomberg Code','Benchmark','Financing Spread (bps)','Remaining Qty(Trade Date)']]
        df.columns=['symbol','benchmark','spread','qty']
        df['symbol']=df['symbol'].astype(str).map(lambda x: x.replace(" Equity","").replace(" EQUITY",""))
        df['spread']=df['spread']*df['qty'].map(lambda x:np.sign(x))
        df['spread']=df['spread']/10000
        df['benchmark']=df['benchmark'].map(lambda x: x.replace('1MHIBOR','HIBOR1M').replace('FIXED','') if isinstance(x,str) else x)
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']]
        return self.financing

In [13]:
class Gf(Broker): 
    
    def get_margin(self):
        df=pd.read_excel(self.mPath, sheet_name='1',skiprows=13)
        #df=df[df["Bloomberg Code"].notna()]
        df=df.loc[:,['Underlying BB Ticker','IA %']]
        df.columns=['symbol','margin']
        df['symbol']=df['symbol'].map(lambda x: x.replace(" Equity",""))
        df['margin']=df['margin']/-100        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):   
        df=pd.read_excel(self.vPath, sheet_name='1',skiprows=13)
        df=df.loc[:,['Underlying BB Ticker','Funding Benchmark','Funding Spread','Quantity','Client Direction']]
        df.columns=['symbol','benchmark','spread','qty','side']
        df['side']=df['side'].map({'Long':1,'Short':-1})
        df['qty']=df['qty']*df['side']
        df['symbol']=df['symbol'].map(lambda x: x.replace(" Equity",""))
        df['spread']=df['spread']/10000
        df['benchmark']=df['benchmark'].map(lambda x: x.replace('USD.SOFR.1D','SOFRON').replace('HKD.HIBOR.1D','HIBORON') if isinstance(x,str) else x)
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']] 
        return self.financing

In [14]:
def ht_symbol(ticker):
    ticker=ticker.replace('.',' ').replace(" SH",' CH').replace("SZ",'CH').replace(" O",' US').replace(" N",' US').replace(' Equity',"")
    if "HK" in ticker:
        ticker=ticker.lstrip("0")
    return ticker

class Ht(Broker):
    
    def get_margin(self):
        df=pd.read_excel(self.mPath, sheet_name='标的明细',skiprows=1)
        #df=df[df["Bloomberg Code"].notna()]
        df=df[df['分红调整剩余数量']!=0]
        df=df.loc[:,['标的代码','最低比例']]
        df.columns=['symbol','margin']        
        df['symbol']=df['symbol'].map(lambda x: x.replace(" Equity",""))        
        self.margin=self.margin_summary(df)
        return self.margin
        df['side']=df['side'].map({'Long':1,'Short':-1})
        df['qty']=df['qty']*df['side']
    def get_financing(self):   
        df=pd.read_excel(self.vPath, sheet_name='标的明细',skiprows=1)
        df=df.loc[:,['标的代码','参考浮动利率','利差','分红调整剩余数量','客户头寸方向']]
        df.columns=['symbol','benchmark','spread','qty','side']
        df=df[df['qty']!=0]  
        df['side']=df['side'].map({'多头':1,'空头':-1})
        df['qty']=df['qty']*df['side']        
        df['symbol']=df['symbol'].map(ht_symbol)
        df['benchmark']=df['benchmark'].map(lambda x: x.replace('SOFR.IR','SOFRON').replace('HIBOR1M.IR','HIBOR1M').replace('HIBORON.IR','HIBORON') if isinstance(x,str) else x)
        df['broker']="Haitong"
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']] 
        return self.financing
    

In [15]:
class Huatai(Broker): 
    
    def get_margin(self):
        df=pd.read_excel(self.mPath, sheet_name='Position Details')
        #df=df[df["Bloomberg Code"].notna()]
        df=df.loc[:,['Underlying','InitialMarginRatio']]
        df.columns=['symbol','margin']
        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_excel(self.mPath, sheet_name='Position Details')
        #df=df[df[\"Bloomberg Code\"].notna()]
        df=df.loc[:,['Underlying','RefBenchmark','Spread','Position']]
        df.columns=['symbol','benchmark','spread','qty']
        df['symbol']=df['symbol'].map(lambda x: x.replace(" Equity",""))
        df['spread']=df['spread']/10000
        df['benchmark']=df['benchmark'].map(lambda x: x.replace('SOFR1D','SOFRON').replace('1M HIBOR','HIBOR1M').replace('HOISHKD','HIBORON') if isinstance(x,str) else x)
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']] 
        return self.financing

In [16]:
def exch_to_code(symbol):
    exchDict={
        "TAI":"TT",
        "HKE":'HK',
        "SHG":'C1',
        'NAS':"US",
        'NYS':"US",
        "ARC":"US",
        'ROCO':"TT",
        'KRX':"KS",
        'ASX':"AU",
        'SGX':"SP",
        'SHE':"C2",
        'ZK':'CH'
        
    }
    exch=symbol.split(".")[1]
    exch_code=exchDict[exch]
    symbol=symbol.replace(exch,exch_code).replace("."," ")
    if "HK" in symbol:
        symbol=symbol.lstrip("0")
    if "KS" in symbol:
        symbol=symbol.zfill(9)
    if "C2" in symbol:
        symbol=symbol.zfill(9)
    if '929 TT' in symbol:
        symbol='00929 TT'
    if 'BS6 SP' in symbol:
        symbol='YZJSGD SP'
    return symbol

In [17]:
class Cimb(Broker):  
    
    def get_margin(self):
        df=pd.read_csv(self.mPath)
        df=df.loc[:,['SecurityID','InitialMargin%']]
        df.columns=['symbol','margin']        
        df['symbol']=df['symbol'].map(lambda x:exch_to_code(x))
        df['margin']=df['margin'].map(lambda x: float(x.replace('%',"") if isinstance(x,str) else x*100))
        df['margin']=df['margin']/100        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_csv(self.mPath)
        #df=df[df[\"Bloomberg Code\"].notna()]
        df=df.loc[:,['SecurityID','BenchmarkRate','MarkupRate','NumberOfShares Units','Long Short']]
        df.columns=['symbol','benchmark','spread','qty','side']
        df['side']=df['side'].map({'LONG':1,'SHORT':-1})
        df['symbol']=df['symbol'].map(lambda x:exch_to_code(x))
        df['spread']=df['spread']/100*df['side']
        df['qty']=df['qty']*df['side']
        #df['benchmark']=df['benchmark'].map(lambda x: x.replace('SOFR1D','SOFRON').replace('1M HIBOR','HIBOR1M').replace('HOISHKD','HIBORON') if isinstance(x,str) else x)
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']] 
        return self.financing    

In [18]:
def mssg_exch_to_code(exch):
    exchDict={
        'SHH':'CH',
        'HKG':'HK',
        'HKE':'HK',
        'SES':'SP',
        'NSM':'US',
        'PSE':'US',
        "TYO":"JP",
        'NYS':'US',
        'TWO':'TT',
        'TW':'TT',
        'TAI':'TT',
        'NMS':'US'
    }
    exch_code=exchDict[exch]
    return exch_code

def mssg_symbol(symbol):
    if 'BS6' in symbol:
        symbol='YZJSGD'
    return symbol

In [19]:
def ric_to_code(symbol):
    exchDict={
        "HK":'HK',
        "SS":'C1',
        "SH":'CH',
        'SZ':'C2',
        'ZK':'CH',
        'OQ':"US",
        'N':"US",
        'P':"US",
        "T":"JP",
        'SI':'SP',
        'TWO':'TT',
        'TW':'TT',
        'TAI':'TT'
    }
    exch=symbol.split(".")[1]
    exch_code=exchDict[exch]
    symbol=symbol.split(".")[0]+" "+symbol.split(".")[1].replace(exch,exch_code)
    if "HK" in symbol:
        symbol=symbol.lstrip("0")
    if 'YAZG SP' in symbol:
        symbol='YZJSGD SP'
    return symbol

In [20]:
class Mssg(Broker):
    
    def get_margin(self):
        df=pd.read_csv(self.mPath)
        #df['Exchange']=df['Exchange'].map(lambda x:mssg_exch_to_code(x))
        #df['Security']=df['Security'].map(lambda x:mssg_symbol(x))
        df['symbol']=df['RIC'].map(lambda x: ric_to_code(x))
        df=df.loc[:,['symbol','Margin_Perc']]
        df.columns=['symbol','margin']
        df=df[df['margin'].notna()]
        df['margin']=df['margin']/100        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_csv(self.vPath)
        df['Exchange']=df['Exchange'].map(lambda x:mssg_exch_to_code(x))
        df['Security']=df['Security_Code'].map(lambda x:mssg_symbol(x))
        #df['symbol']=df['RIC'].map(lambda x: ric_to_code(x))
        df['Security']=df['Security']+" "+df['Exchange']
        df=df.loc[:,['Security','CurrCode','EffectiveRate','TradeVolumn']]
        df.columns=['symbol','benchmark','spread','qty']
        df['spread']=df['spread']/-100-0.005
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']] 
        return self.financing      

In [21]:
def jefs_symbol(symbol):
    symbol=symbol.replace("/"," ").replace(" JT"," JP")
    return symbol
    
class Jefs(Broker):

    def get_margin(self):
        df=pd.read_excel(self.mPath)
        df['symbol']=df['SecondaryTradeRef'].map(jefs_symbol)
        df['margin']=df['LockUp']/df['Notional1']
        df=df.loc[:,['symbol','margin']]
        #df.columns=['symbol','margin']
        df=df[df['margin'].notna()]
        #df['margin']=df['margin']/-100        
        self.margin=self.margin_summary(df)
        return self.margin
    
    def get_financing(self):
        df=pd.read_csv(self.vPath)
        df['symbol']=df['Symbol'].map(jefs_symbol)
        df=df.loc[:,['symbol','Financing Benchmark','Spread','Quantity']]
        df.columns=['symbol','benchmark','spread','qty']
        df['spread']=df['spread']/100
        df['broker']=self.name
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']] 
        return self.financing      

In [22]:
def jeff_num_cov(amount):
    amount=amount.replace(",","")
    sign=1
    if "(" in amount:
        amount=amount.replace("(","").replace(")","")
        sign=-1
    amount=float(amount)*sign
    return amount

In [23]:
class Jefc(Broker):

    def get_margin(self):
        with pdfplumber.open(self.mPath) as pdf:
            textline2=['']
            for page in pdf.pages:
                page02 = page.extract_text().split("\n")[4:-2] #指定页码
                textline2 = textline2+page02
        textline2
        index01=textline2.index('Single Position')
        
        for i in range(index01,len(textline2)):
            if 'Total Single Position' in textline2[i]:
                index02=i
                break
        entryList=[]
        if "Non Marginable Securities" in textline2:
            index00=textline2.index('Non Marginable Securities')
            for k in range(index00,index01):
                if 'FX Rate' in textline2[k]:
                    entryList.append(textline2[k+1])          
        for j in range(index01,index02):
            if 'FX Rate' in textline2[j]:
                entryList.append(textline2[j+1])
        entryform2=[]
        for entry in entryList:
            entryitems=entry.split(' ')
            comb=['a']
            comb[0]=entryitems[0]
            comb=comb+entryitems[-2:-1]
            entryform2.append(comb)
        columns2=['symbol','margin']
        df=pd.DataFrame(entryform2, columns=columns2)
        df['margin']=df['margin'].astype(float)/100
        df['symbol']=df['symbol'].map(lambda x: x+" US" if len(x)==4 else sedol_dict[x])
        self.margin=self.margin_summary(df)        
        return self.margin 

    def get_financing(self):
        with pdfplumber.open(self.vPath) as pdf:
            textline3=['']
            for page in pdf.pages:
                page03 = page.extract_text().split("\n")[4:-2] #指定页码
                textline3 = textline3+page03
        
        index1=0
        index2=len(textline3)
        if 'General Collateral' in textline3:
            index1=textline3.index('General Collateral')
        if 'Non-General Collateral' in textline3:
            index2=textline3.index('Non-General Collateral')
        textline3_1=textline3[index1:index2]
        textline3_2=textline3[index2:len(textline3)]
        currencyList=['HKD - Hong Kong Dollar','USD - U.S. Dollars']
        gcList=[]
        for currency in currencyList:
            if currency in textline3_1:
                starter1=textline3_1.index(currency)+1
                for i in range(starter1,len(textline3_1)):
                    if currency in textline3_1[i]:
                        ender1=i
                        gcList.append(textline3_1[starter1:ender1])
                        break
        
        ngcList=[]
        for currency in currencyList:
            if currency in textline3_2:
                starter2=textline3_2.index(currency)+1
                for i in range(starter2,len(textline3_2)):
                    if currency in textline3_2[i]:
                        ender2=i
                        ngcList.append(textline3_2[starter2:ender2])
                        break
        
        borrowList=gcList+ngcList
        borrowList=list(itertools.chain(*borrowList))
        borrowForm=[]
        for entry in borrowList:
            entry=entry.replace(" *","").split(' ')
            if jeff_num_cov(entry[-6])==0:
                qty=entry[-10]
                rate=entry[-5]
            else:
                qty=entry[-11]
                rate=entry[-6]
            cusip=entry[0]            
            borrow=[qty,cusip,rate]
            borrowForm.append(borrow)
        borrowForm
        column=['qty','symbol','total']
        df=pd.DataFrame(borrowForm, columns=column)
        df['symbol']=df['symbol'].map(lambda x: x+" US" if len(x)==4 else sedol_dict[x])
        df['qty']=df['qty'].map(jeff_num_cov)
        df['benchmark']=df['symbol'].map(lambda x: rate_dict[x[-2:]])
        df['broker']='jefc'
        df['spread']=df['total'].map(jeff_num_cov)-df['benchmark']
        df['spread']=df['spread']/100
        self.financing=df.loc[:,['symbol','broker','benchmark','spread','qty']]
        return self.financing 

In [24]:
MACQ=Macq('Macquarie','PsPositionReport')
CLSA=Clsa('CLSA','MGN-SeahawkChinaDyn-721088','VAL-SeahawkChinaDyn-721904_721906')
CICC=Cicc('CICC','Seahawk CDF_ValuationReport')
GF=Gf('GF','CRIC_SEAHAWK')
HT=Ht('HT','Client_ValuationReportNew_2')
HUATAI=Huatai('Huatai','Valuation Report')
CIMB=Cimb('CIMB','dailyopenpositions')
MSSG=Mssg('MSSG','LP0139007A_POSITION','LP0139007A_EXTRABORROWCOST')
JEFS=Jefs('Jefferies','Exposure_Trades','P1741_SwapFinancingSummary')
JEFC=Jefc('Jefferies','Portfolio-Margin-Detail','Rebate-Summary-Local-Currency')

In [25]:
brokerList=[MACQ,CLSA,CICC,GF,HT,HUATAI,CIMB,MSSG,JEFS,JEFC]

In [26]:
marginSummary=pd.DataFrame(columns=['symbol'])
for broker in brokerList:
    brokerMargin=broker.get_margin()
    marginSummary=pd.merge(marginSummary,brokerMargin,on='symbol',how='outer')
marginSummary.sort_values(by='symbol', inplace=True)
#marginSummary

C:\Users\Ning\anaconda3\envs\seahawk\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Ning\anaconda3\envs\seahawk\lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [27]:
financingSummary=pd.DataFrame(columns=['symbol'])
for broker in brokerList:
    brokerFinancing=broker.financing_summary(broker.get_financing())
    financingSummary=pd.merge(financingSummary,brokerFinancing,on='symbol',how='outer')
financingSummary.sort_values(by='symbol', inplace=True)
#financingSummary

C:\Users\Ning\anaconda3\envs\seahawk\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Ning\anaconda3\envs\seahawk\lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [28]:
financingDetails=pd.DataFrame(columns=['symbol','broker','benchmark','spread'])
for broker in brokerList:
    brokerFinancingDetails=broker.get_financing()
    financingDetails=pd.concat([financingDetails,brokerFinancingDetails])
financingDetails.sort_values(by=['symbol','broker'], inplace=True)
financingDetails=financingDetails[financingDetails['qty']!=0]
#financingDetails

C:\Users\Ning\AppData\Local\Temp\ipykernel_23460\4046969.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  financingDetails=pd.concat([financingDetails,brokerFinancingDetails])
C:\Users\Ning\anaconda3\envs\seahawk\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Ning\anaconda3\envs\seahawk\lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [29]:
summaryPath="margin_financing_summary_{}.xlsx".format(datetime.now().strftime("%Y%m%d"))
writer=pd.ExcelWriter(summaryPath)
marginSummary.to_excel(writer, sheet_name='margin',index=False)
financingSummary.to_excel(writer, sheet_name='financingSummary',index=False)
financingDetails.to_excel(writer, sheet_name='financingDetails',index=False)
writer.close()

In [30]:
os.system(f'start excel \"{summaryPath}\"')

0